# Import Library

In [ ]:
# Import library
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Define File Path

In [ ]:
blogs_data = f'/content/drive/MyDrive/blog/Medium Blog Data.csv'
ratings_data = f'/content/drive/MyDrive/blog/Blog Ratings.csv'
authors_data = f'/content/drive/MyDrive/blog/Author Data.csv'

# Load File

In [ ]:
blogs = pd.read_csv(blogs_data,sep=',')
ratings = pd.read_csv(ratings_data,sep=',')
authors = pd.read_csv(authors_data,sep=',')

# Data Understanding

Dataset ini berisi informasi blog yang terdiri dari 3 filr csv yaitu



*   Medium Blog Data.csv yang berisi informasi data blog
*   Blog Ratings.csv yang berisi data penilaian blog oleh pengguna
*   Author Sata.csv yang berisi data penulis blog



In [ ]:
print('Jumlah data blog: ', len(blogs.blog_id.unique()))
print('Jumlah data penilaian blog yang diberikan pengguna: ', len(ratings.userId.unique()))
print('Jumlah data penulis: ', len(authors.author_id.unique()))

# Univariate Exploratory Data Analysis

## Blog

In [ ]:
blogs.info()

In [ ]:
print('Jumlah data blog: ', len(blogs.blog_id.unique()))
print('Jumlah data penulis: ', len(blogs.author_id.unique()))
print('JUmlah data judul: ', len(blogs.blog_title.unique()))
print('Jumlah data konten: ', len(blogs.blog_content.unique()))
print('Jumlah data topik ', len(blogs.topic.unique()))

## Ratings

In [ ]:
ratings.info()

In [ ]:
print('Jumlah data blog id: ', len(ratings.blog_id.unique()))
print('Jumlah data user id: ', len(ratings.userId.unique()))
print('Jumlah data rating: ', len(ratings.ratings))

## Author

In [ ]:
authors.info()

In [ ]:
print('Jumlah data id penulis: ', len(authors.author_id.unique()))
print('Jumlah data penulis: ', len(authors.author_name.unique()))

# Data Preprocessing

##  Menggabungkan dataframe ratings dengan blogs

In [ ]:
# Menggabungkan dataframe ratings dengan blogs berdasarkan nilai blog_id
blog = pd.merge(ratings, blogs , on='blog_id', how='left')
blog

In [ ]:
# Menghitung jumlah Book-Rating kemudian menggabungkannya berdasarkan blog_id
blog.groupby('blog_id').sum()

## Menggabungkan ratings dengan dataframe blog

In [ ]:
# Menggabungkan ratings dengan dataframe blog berdasarkan blog_id
all_blog = pd.merge(ratings, blogs[['blog_id','blog_title', 'topic']], on='blog_id', how='left')

# Print dataframe all_blog_name
all_blog

In [ ]:
print(all_blog.head().to_markdown())

## Mengecek missing value pada dataframe all_blog

In [ ]:
# Mengecek missing value pada dataframe all_blog
all_blog.isnull().sum()

## Random Sampling

In [ ]:
# jumlah sampel
num_samples = 1000

# Perform random sampling
sampled_all_blog = all_blog.sample(n=num_samples, random_state=42)

## Data Preparation

## Mengecek kembali missing value pada variabel all_blog_clean

In [ ]:
# Mengecek kembali missing value pada variabel all_blog_clean
sampled_all_blog.isnull().sum()

## kategori topic yang unik

In [ ]:
# Mengecek kategori topic yang unik
sampled_all_blog.topic.unique()

## Membuat variabel preparation yang berisi dataframe all_clean_blog

In [ ]:
# Membuat variabel preparation yang berisi dataframe all_clean_blog kemudian mengurutkan berdasarkan blog
preparation = sampled_all_blog
preparation.sort_values('blog_id')

## Mengonversi data series menjadi dalam bentuk list

In [ ]:
# Mengonversi data series ‘blog_id’ menjadi dalam bentuk list
id = preparation['blog_id'].tolist()

# Mengonversi data series ‘blog_title’ menjadi dalam bentuk list
blog_title = preparation['blog_title'].tolist()

# Mengonversi data series ‘topicr’ menjadi dalam bentuk list
topic = preparation['topic'].tolist()

print(len(id))
print(len(blog_title))
print(len(topic))

## Membuat dictionary untuk data

In [ ]:
# Membuat dictionary untuk data 'id', 'blog_title', dan 'topic'
blog_new = pd.DataFrame({
    'id': id,
    'title': blog_title,
    'topic': topic
})
blog_new

# Model Development dengan Content Based Filtering

In [ ]:
data = blog_new
data.sample(5)

## TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()

# Melakukan perhitungan idf pada data topic
tf.fit(data['topic'])

# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names_out()

## Melakukan fit lalu ditransformasika

In [ ]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data['topic'])

# Melihat ukuran matrix tfidf
tfidf_matrix.shape

## Mengubah vektor tf-idf

In [ ]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

In [ ]:
# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan topic
# Baris diisi dengan title

pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names_out(),
    index=data.title
).sample(22, axis=1).sample(10, axis=0)

## Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

In [ ]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa judul blog
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['title'], columns=data['title'])
print('Shape:', cosine_sim_df.shape)

# Melihat similarity matrix pada setiap blog
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

## Create Prediction Function

In [ ]:
def blog_recommendations(title, similarity_data=cosine_sim_df, items=data[['title', 'topic']], k=5):
    """
    Rekomendasi blog berdasarkan kemiripan dataframe

    Parameter:
    ---
    title : tipe data string (str)
                Judul Blog (index kemiripan dataframe)
    similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan blog sebagai
                      indeks dan kolom
    items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
    k : tipe data integer (int)
        Banyaknya jumlah rekomendasi yang diberikan
    ---


    Pada index ini, kita mengambil k dengan nilai similarity terbesar
    pada index matrix yang diberikan (i).
    """


    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,title].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    # Drop title agar judul blog yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(title, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

## Example Title 1

In [ ]:
data[data.title.eq('Tasha’s Trinkets')]

In [ ]:
# Mendapatkan rekomendasi blog yang mirip dengan Coming soon: Tasha’s Trinkets
blog_recommendations('Tasha’s Trinkets')

## Example Title 2

In [ ]:
data[data.title.eq('EVE Ignites NFT Hope')]

In [ ]:
# Mendapatkan rekomendasi blog yang mirip dengan Coming soon: How to Emboss Images using Python'
blog_recommendations('How to Emboss Images using Python')

## Example Title 3

In [ ]:
data[data.title.eq('EVE Ignites NFT Hope')]

In [ ]:
# Mendapatkan rekomendasi blog yang mirip dengan EVE Ignites NFT Hope
blog_recommendations('EVE Ignites NFT Hope')